In [2]:
DRIVE = False
if DRIVE:
  from google.colab import drive
  drive.mount('/content/drive')

import torch

In [ ]:
import statistics

def run_average(perm, path, exp_names, verbose, num_runs):
  total = {}
  actual_task = {"0": [], "1": [], "2": [], "3": [], "4": []}
  learning_pos = {"0": [], "1": [], "2": [], "3": [], "4": []}
  for idx_1, exp_name in enumerate(exp_names):
    averages = [0] * num_runs
    if verbose: print(f"experiment {exp_name}")

    for run in range(num_runs):
      if verbose: print(f"Run: {run}")
      inner_averages = []
      learn_speed = torch.load(f'{path}/ls_{exp_name}_{run}_default.pth')
      count = [torch.bincount(learn_speed[i].int()) for i in range(5)]

      # average learning speed:
      for idx_2, t in enumerate(count):
        avg = 0
        for idx_in, el in enumerate(t):
          avg += el * idx_in

        temp = learning_pos[f"{idx_2}"]
        temp.append(float(avg/t.sum()))
        learning_pos[f"{idx_2}"] = temp

        temp = actual_task[f"{perm[idx_1][idx_2]}"]
        temp.append(float(avg/t.sum()))
        actual_task[f"{perm[idx_1][idx_2]}"] = temp

        if verbose: print(f"Learned Task {idx_2} (actual task {perm[idx_1][idx_2]}) avg. learning speed: {avg/t.sum()}")
        inner_averages.append(avg/t.sum())

      # compute speed average for this run
      averages[run] = float(sum(inner_averages) / len(inner_averages))
    for pr_idx, avg in enumerate(averages):
      if verbose: print(f"Run {pr_idx} average: {avg}")

    total[exp_name] = (statistics.mean(averages), statistics.stdev(averages))
  for key in actual_task.keys():
    actual_task[key] = (statistics.mean(actual_task[key]), statistics.stdev(actual_task[key]))
    learning_pos[key] = (statistics.mean(learning_pos[key]), statistics.stdev(learning_pos[key]))


  return total, actual_task, learning_pos

In [ ]:
"""
Here now print results when ordering:
tot: contains for each exp the average learning speed and std.
at:  contains for each task the average learning speed for all the runs and exps (75 total vals)
lp:  contains for each learning position the average learning speed (75 total val, each actual tast 15 times)
"""

exp_permutations = [[0,1,2,3,4],[4,3,1,2,0],[1,4,3,0,2],[3,2,0,4,1],[2,0,4,1,3]]
exp_names = ['p0','p1','p2','p3','p4']
path = "/content/drive/MyDrive/exp_results/default" # TODO: exchange with own path, example path to drive
verbose = True
num_runs = 15

tot, at, lp = run_average(exp_permutations, path, exp_names, verbose, num_runs)

In [ ]:
tot, at, lp

({'p0': (9.181870651245116, 0.030117535857021795),
  'p1': (9.135250854492188, 0.04727069689023117),
  'p2': (9.085728073120118, 0.04368395110567139),
  'p3': (9.182953389485677, 0.04310841666885159),
  'p4': (9.155961227416991, 0.04015393934799925)},
 {'0': (9.239983990987142, 0.4366503540223478),
  '1': (9.051838658650716, 0.24612615918127856),
  '2': (9.258153381347656, 0.5331773322864435),
  '3': (9.111469294230144, 0.43216376710790794),
  '4': (9.08031867980957, 0.32653002683095234)},
 {'0': (9.414119987487792, 0.22593978638442108),
  '1': (8.53191869099935, 0.21690041256063067),
  '2': (8.969469337463378, 0.26041009284384103),
  '3': (9.41155865987142, 0.1810954725436557),
  '4': (9.414697329203287, 0.18758244565359192)})